In [1]:
import sys
sys.path.append("./")
import numpy as np
from quaos.basic_functions import int_to_bases, bases_to_int,complex_phase_value
from quaos.paulis import (PauliSum, PauliString, Pauli,)
import sympy as sym
from quaos.circuits import Gate, Circuit, Hadamard as H, SUM as CX, PHASE as S

In [10]:
def matrix_to_string(matrix):
    """Convert a SymPy matrix with 0s and 1s into a string."""
    return ''.join(map(str, list(map(int, matrix.tolist()[0]))))

In [19]:
# all two qubit 4 Pauli hamiltonians
r11, r12, r21, r22, r31, r32, r41, r42 = sym.symbols('r11 r12 r21 r22 r31 r32 r41 r42')
s11, s12, s21, s22, s31, s32, s41, s42 = sym.symbols('s11 s12 s21 s22 s31 s32 s41 s42')

M = sym.Matrix([[r11, r12,s11, s12],[r21, r22, s21, s22], [r31, r32, s31, s32], [r41, r42, s41, s42]])
J = sym.Matrix([[0, 0, 1, 0],[0, 0, 0, 1], [1, 0, 0, 0], [0, 1, 0, 0]])
#H = sym.Matrix([[0, 0, 1, 0],[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1]])

dims = [2,2]
n_paulis = 4
pauli_string_max_options = np.prod([2,2,2,2])
#print(pauli_string_max_options)
coefficients = [1 for i in range(n_paulis)]
for i in range(pauli_string_max_options):
    for j in range(pauli_string_max_options):
        if i!= j:
            for k in range(pauli_string_max_options):
                if k != i and k != j:
                    for l in range(pauli_string_max_options):
                        if l != i and l != j and l != k:
                            pauli_strings = ['' for i in range(n_paulis)]
                            exponents0 = int_to_bases(i, [2,2,2,2])
                            for m in range(2):
                                r, s = int(exponents0[2*m]), int(exponents0[2*m+1])
                                pauli_strings[0] += f"x{r}z{s} "
                            exponents1 = int_to_bases(j, [2,2,2,2])
                            for m in range(2):
                                r, s = int(exponents1[2*m]), int(exponents1[2*m+1])
                                pauli_strings[1] += f"x{r}z{s} "
                            exponents2 = int_to_bases(k, [2,2,2,2])
                            for m in range(2):
                                r, s = int(exponents2[2*m]), int(exponents2[2*m+1])
                                pauli_strings[2] += f"x{r}z{s} "
                            exponents3 = int_to_bases(l, [2,2,2,2])
                            for m in range(2):
                                r, s = int(exponents3[2*m]), int(exponents3[2*m+1])
                                pauli_strings[3] += f"x{r}z{s}"
                            #print(pauli_strings)
                            P = PauliSum(pauli_strings, weights=coefficients ,dimensions=dims, phases=None, standardise=False)
                            print('PauliSum:')
                            print(P)
                            P_sym = sym.Matrix(P.symplectic())

                            # P_c
                            P_c = (M*P_sym.T).T
                            #H_P_c = P_c*H.T

                            # Boundary conditions
                            bounds = M.T * J * M - J
                            #display(bounds)
                            C = Circuit(dims,gates=[H(0,2)])
                            # Plug in variables to find minimum of cost function
                            n_opts = [1632,2448,6273,9282,16932,24582,33048,36873]
                            for n in n_opts:
                                if n % 1000 == 0:
                                    print(n)
                                #n = 1632
                                vars = int_to_bases(n, [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2])
                                subs_bounds = bounds.subs({r11: vars[0],r12: vars[1],r21: vars[2],r22: vars[3],r31: vars[4],r32: vars[5],r41: vars[6],r42: vars[7],s11: vars[8],s12: vars[9],s21: vars[10],s22: vars[11],s31: vars[12],s32: vars[13],s41: vars[14],s42: vars[15]})
                                #display(subs_bounds)
                                if subs_bounds.equals(sym.Matrix.zeros(subs_bounds.rows, subs_bounds.cols)):
                                    subs_P_c = P_c.subs({r11: vars[0],r12: vars[1],r21: vars[2],r22: vars[3],r31: vars[4],r32: vars[5],r41: vars[6],r42: vars[7],s11: vars[8],s12: vars[9],s21: vars[10],s22: vars[11],s31: vars[12],s32: vars[13],s41: vars[14],s42: vars[15]})
                                    subs_P_c_PS = PauliSum([PauliString(np.array(subs_P_c[i,0:2].tolist()[0],dtype=np.int32),np.array(subs_P_c[i,2:4].tolist()[0],dtype=np.int32),dimensions=dims) for i in range(4)],dimensions=dims, phases=None, standardise=False)
                                    #print(subs_P_c_PS)
                                    H_P_c_PS = C.act(subs_P_c_PS)
                                    #print(H_P_c_PS)
                                    subs_H_P_c = sym.Matrix(np.array(H_P_c_PS.symplectic(),dtype=np.int32))
                                    #display(subs_H_P_c)

                                    dictionary = {matrix_to_string(subs_P_c[0,:]):-coefficients[0], matrix_to_string(subs_P_c[1,:]):-coefficients[1], matrix_to_string(subs_P_c[2,:]):-coefficients[2], matrix_to_string(subs_P_c[3,:]):-coefficients[3]}
                                    #display(dictionary)
                                    if matrix_to_string(subs_H_P_c[0,:]) not in dictionary:
                                        dictionary[matrix_to_string(subs_H_P_c[0,:])] = 0
                                    if matrix_to_string(subs_H_P_c[1,:]) not in dictionary:
                                        dictionary[matrix_to_string(subs_H_P_c[1,:])] = 0
                                    if matrix_to_string(subs_H_P_c[2,:]) not in dictionary:
                                        dictionary[matrix_to_string(subs_H_P_c[2,:])] = 0
                                    if matrix_to_string(subs_H_P_c[3,:]) not in dictionary:
                                        dictionary[matrix_to_string(subs_H_P_c[3,:])] = 0
                                    dictionary[matrix_to_string(subs_H_P_c[0,:])] += int(complex_phase_value(H_P_c_PS.phases[0],2).real)*coefficients[0]
                                    dictionary[matrix_to_string(subs_H_P_c[1,:])] += int(complex_phase_value(H_P_c_PS.phases[1],2).real)*coefficients[1]
                                    dictionary[matrix_to_string(subs_H_P_c[2,:])] += int(complex_phase_value(H_P_c_PS.phases[2],2).real)*coefficients[2]
                                    dictionary[matrix_to_string(subs_H_P_c[3,:])] += int(complex_phase_value(H_P_c_PS.phases[3],2).real)*coefficients[3]
                                    #display(dictionary)
                                    cost = sum(abs(value) for value in dictionary.values())
                                    #print(n,cost, dictionary)
                                    if cost == 0:
                                        print('H sym')
                                        print('Symmetrized PauliSum:')
                                        print(H_P_c_PS)
                                        subs_M = M.subs({r11: vars[0],r12: vars[1],r21: vars[2],r22: vars[3],r31: vars[4],r32: vars[5],r41: vars[6],r42: vars[7],s11: vars[8],s12: vars[9],s21: vars[10],s22: vars[11],s31: vars[12],s32: vars[13],s41: vars[14],s42: vars[15]})
                                        #display(subs_M)
                                        print()
                                        print()
                                        break
                            if cost != 0:
                                print('No H Sym')
                                print()
                                print()

                            

PauliSum:
(1+0j)|x0z0 x0z0 | 0 
(1+0j)|x0z0 x0z1 | 0 
(1+0j)|x0z0 x1z0 | 0 
(1+0j)|x0z0 x1z1 | 0 

H sym
Symmetrized PauliSum:
(1+0j)|x0z0 x0z0 | 0 
(1+0j)|x0z0 x1z0 | 0 
(1+0j)|x0z0 x0z1 | 0 
(1+0j)|x0z0 x1z1 | 0 



PauliSum:
(1+0j)|x0z0 x0z0 | 0 
(1+0j)|x0z0 x0z1 | 0 
(1+0j)|x0z0 x1z0 | 0 
(1+0j)|x0z1 x0z0 | 0 

H sym
Symmetrized PauliSum:
(1+0j)|x0z0 x0z0 | 0 
(1+0j)|x0z1 x0z0 | 0 
(1+0j)|x1z0 x0z0 | 0 
(1+0j)|x0z0 x1z0 | 0 



PauliSum:
(1+0j)|x0z0 x0z0 | 0 
(1+0j)|x0z0 x0z1 | 0 
(1+0j)|x0z0 x1z0 | 0 
(1+0j)|x0z1 x0z1 | 0 

No H Sym


PauliSum:
(1+0j)|x0z0 x0z0 | 0 
(1+0j)|x0z0 x0z1 | 0 
(1+0j)|x0z0 x1z0 | 0 
(1+0j)|x0z1 x1z0 | 0 

No H Sym


PauliSum:
(1+0j)|x0z0 x0z0 | 0 
(1+0j)|x0z0 x0z1 | 0 
(1+0j)|x0z0 x1z0 | 0 
(1+0j)|x0z1 x1z1 | 0 

No H Sym


PauliSum:
(1+0j)|x0z0 x0z0 | 0 
(1+0j)|x0z0 x0z1 | 0 
(1+0j)|x0z0 x1z0 | 0 
(1+0j)|x1z0 x0z0 | 0 

H sym
Symmetrized PauliSum:
(1+0j)|x0z0 x0z0 | 0 
(1+0j)|x0z1 x0z0 | 0 
(1+0j)|x1z0 x0z0 | 0 
(1+0j)|x0z0 x0z1 | 0 



PauliSum:
(1+0j)

KeyboardInterrupt: 

In [54]:
# all two qubit 4 Pauli hamiltonians
r11, r12, r21, r22, r31, r32, r41, r42 = sym.symbols('r11 r12 r21 r22 r31 r32 r41 r42')
s11, s12, s21, s22, s31, s32, s41, s42 = sym.symbols('s11 s12 s21 s22 s31 s32 s41 s42')

M = sym.Matrix([[r11, r12,s11, s12],[r21, r22, s21, s22], [r31, r32, s31, s32], [r41, r42, s41, s42]])
J = sym.Matrix([[0, 0, 1, 0],[0, 0, 0, 1], [1, 0, 0, 0], [0, 1, 0, 0]])
#H = sym.Matrix([[0, 0, 1, 0],[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1]])

dims = [2,2]
n_paulis = 4
pauli_string_max_options = np.prod([2,2,2,2])
#print(pauli_string_max_options)
coefficients = [1 for i in range(n_paulis)]

i = 0
j = 1
k = 2
l = 3
pauli_strings = ['' for i in range(n_paulis)]
exponents0 = int_to_bases(i, [2,2,2,2])
for m in range(2):
    r, s = int(exponents0[2*m]), int(exponents0[2*m+1])
    pauli_strings[0] += f"x{r}z{s} "
exponents1 = int_to_bases(j, [2,2,2,2])
for m in range(2):
    r, s = int(exponents1[2*m]), int(exponents1[2*m+1])
    pauli_strings[1] += f"x{r}z{s} "
exponents2 = int_to_bases(k, [2,2,2,2])
for m in range(2):
    r, s = int(exponents2[2*m]), int(exponents2[2*m+1])
    pauli_strings[2] += f"x{r}z{s} "
exponents3 = int_to_bases(l, [2,2,2,2])
for m in range(2):
    r, s = int(exponents3[2*m]), int(exponents3[2*m+1])
    pauli_strings[3] += f"x{r}z{s}"
#print(pauli_strings)
P = PauliSum(pauli_strings, weights=coefficients ,dimensions=dims, phases=None, standardise=False)
P_sym = sym.Matrix(np.array(P.symplectic(),dtype=np.int32))
display(P_sym)
#display(P_sym*H.T)

# P_c
P_c = (M*P_sym.T).T
#H_P_c = P_c*H.T
#display(P_c)
#display(P_c*H.T)



# Boundary conditions
bounds = M.T * J * M - J
#display(bounds)
C = Circuit(dims,gates=[H(0,2)])
# Plug in variables to find minimum of cost function
for n in range(2**16):
    if n % 1000 == 0:
        print(n)
    #n = 1632
    vars = int_to_bases(n, [2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2])
    subs_bounds = bounds.subs({r11: vars[0],r12: vars[1],r21: vars[2],r22: vars[3],r31: vars[4],r32: vars[5],r41: vars[6],r42: vars[7],s11: vars[8],s12: vars[9],s21: vars[10],s22: vars[11],s31: vars[12],s32: vars[13],s41: vars[14],s42: vars[15]})
    #display(subs_bounds)
    if subs_bounds.equals(sym.Matrix.zeros(subs_bounds.rows, subs_bounds.cols)):
        #print(n)
        #subs_H_P_c = H_P_c.subs({r11: vars[0],r12: vars[1],r21: vars[2],r22: vars[3],r31: vars[4],r32: vars[5],r41: vars[6],r42: vars[7],s11: vars[8],s12: vars[9],s21: vars[10],s22: vars[11],s31: vars[12],s32: vars[13],s41: vars[14],s42: vars[15]})
        #print(subs_P_c[0,0:2].tolist()[0],subs_P_c[0,2:4].tolist())
        #print(type(np.array(subs_P_c[0,0:2].tolist()[0])))
        #print(PauliString(np.array(subs_P_c[0,0:2].tolist()[0]),np.array(subs_P_c[0,2:4].tolist()[0]),dimensions=dims))
        subs_P_c = P_c.subs({r11: vars[0],r12: vars[1],r21: vars[2],r22: vars[3],r31: vars[4],r32: vars[5],r41: vars[6],r42: vars[7],s11: vars[8],s12: vars[9],s21: vars[10],s22: vars[11],s31: vars[12],s32: vars[13],s41: vars[14],s42: vars[15]})
        subs_P_c_PS = PauliSum([PauliString(np.array(subs_P_c[i,0:2].tolist()[0]),np.array(subs_P_c[i,2:4].tolist()[0]),dimensions=dims) for i in range(4)],dimensions=dims, phases=None, standardise=False)
        H_P_c_PS = C.act(subs_P_c_PS)
        print(H_P_c_PS)
        subs_H_P_c = sym.Matrix(np.array(H_P_c_PS.symplectic(),dtype=np.int32))

        dictionary = {matrix_to_string(subs_P_c[0,:]):-coefficients[0], matrix_to_string(subs_P_c[1,:]):-coefficients[1], matrix_to_string(subs_P_c[2,:]):-coefficients[2], matrix_to_string(subs_P_c[3,:]):-coefficients[3]}
        #display(dictionary)
        dictionary[matrix_to_string(subs_H_P_c[0,:])] += complex_phase_value(H_P_c_PS.phases[0],2)*coefficients[0]
        dictionary[matrix_to_string(subs_H_P_c[1,:])] += complex_phase_value(H_P_c_PS.phases[1],2)*coefficients[1]
        dictionary[matrix_to_string(subs_H_P_c[2,:])] += complex_phase_value(H_P_c_PS.phases[2],2)*coefficients[2]
        dictionary[matrix_to_string(subs_H_P_c[3,:])] += complex_phase_value(H_P_c_PS.phases[3],2)*coefficients[3]
        #display(dictionary)
        cost = sum(value for value in dictionary.values())
        print(n,cost, dictionary)
        if cost == 0:
            display(subs_P_c)
            subs_M = M.subs({r11: vars[0],r12: vars[1],r21: vars[2],r22: vars[3],r31: vars[4],r32: vars[5],r41: vars[6],r42: vars[7],s11: vars[8],s12: vars[9],s21: vars[10],s22: vars[11],s31: vars[12],s32: vars[13],s41: vars[14],s42: vars[15]})
            display(subs_M)
            break
#print(cost)
#display(subs_P_c)
#display(subs_H_P_c[1,:])
#display(subs_H_P_c)
#display(subs_bounds)


Matrix([
[0, 0, 0, 0],
[0, 0, 0, 1],
[0, 1, 0, 0],
[0, 1, 0, 1]])

0
1000
(1+0j)|x0z0 x0z0 | 0 
(1+0j)|x0z1 x0z0 | 0 
(1+0j)|x1z0 x0z0 | 0 
(1+0j)|x1z1 x0z0 | 1 

1632 (-2+1.2246467991473532e-16j) {'0000': 0j, '1000': 0j, '0010': 0j, '1010': (-2+1.2246467991473532e-16j)}
2000
(1+0j)|x0z0 x0z0 | 0 
(1+0j)|x0z0 x1z0 | 0 
(1+0j)|x0z0 x0z1 | 0 
(1+0j)|x0z0 x1z1 | 0 

2448 0j {'0000': 0j, '0100': 0j, '0001': 0j, '0101': 0j}


Matrix([
[0, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 0, 1],
[0, 1, 0, 1]])

Matrix([
[0, 0, 1, 0],
[0, 0, 0, 1],
[1, 0, 0, 0],
[0, 1, 0, 0]])

In [ ]:
print(type(np.array(subs_P_c[0,0:2].tolist()[0])))
print(type(np.array([0,1])))
P_test = PauliString(np.array(subs_P_c[0,0:2].tolist()[0]),np.array(subs_P_c[0,2:4].tolist()[0]),dimensions=dims)
PS_test = PauliSum([P_test,P_test],dimensions=dims, phases=None, standardise=False)
C.act(PS_test)


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


TypeError: Unsupported type <class 'paulis.pauli_sum.PauliSum'> for Gate.act. Expected Pauli, PauliString or PauliSum.

In [42]:
P_test = PauliSum(['x0z0 x0z0'],dimensions=[2,2],weights=[1],phases=[0,0],standardise=False)
g = H(1,2)
C = Circuit(dims,gates=[g])
C.act(P_test)

In [36]:
def matrix_to_string(matrix):
    """Convert a SymPy matrix with 0s and 1s into a string."""
    return ''.join(map(str, matrix.tolist()[0]))


In [18]:
xs = [0,1,2,3]
''.join(str(x) for x in xs)

'0123'

In [23]:
x, y = sym.symbols('x y')
A = {'0': x, '1': y}
A['0'] += y
A

{'0': x + y, '1': y}

In [48]:
sym.Matrix(np.array([[0, 0, 0, 0],[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]],dtype=np.int32)).is_zero

False

In [52]:
A = sym.Matrix([[0, 0], [0, 0]])

# Check if all elements are zero
A.equals(sym.Matrix.zeros(A.rows, A.cols))


True